I wanted to try out this infilling model.
- More info about the model here: https://huggingface.co/CarperAI/FIM-NeoX-1.3B
- Currently doesn't seem to work, due to issue with the tokenizer, see https://huggingface.co/CarperAI/FIM-NeoX-1.3B/discussions/1
- This model fits perfectly onto an 8gb GPU (nvidia-smi is saying: 7910MiB /  7981MiB)

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [2]:
# Takes ~1.5 minutes to load
tokenizer = AutoTokenizer.from_pretrained("/media/sid/bigdata/models/FIM-NeoX-1.3B")
model = AutoModelForCausalLM.from_pretrained("/media/sid/bigdata/models/FIM-NeoX-1.3B")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print(device)

In [89]:
def infill(prefix, suffix):
    input_ids = (
        torch.tensor(
            [
                50253,
                *tokenizer(suffix)["input_ids"],
                50254,
                *tokenizer(prefix)["input_ids"],
                50255,
            ]
        )
        .reshape(1, -1)
        .to(device)
    )
    attention_mask = torch.ones_like(input_ids)
    infilled = model.generate(
        input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=20
    )
    filled_text = tokenizer.decode(
        infilled[0, ...][input_ids[0].tolist().index(50255) + 1 :], special_tokens=False
    )
    return filled_text


infill("More than half the world's", "resides in North America")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'\n  **Societal**                                                                                                                                                                                                                                                                                                                        '